In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_columns', None)

In [2]:
acs2019_1 = pd.read_csv('../data/productDownload_2021-10-30T134120/ACSDP1Y2019.DP05_data_with_overlays_2021-10-18T135741.csv')

In [3]:
acs2019_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 841 entries, 0 to 840
Columns: 358 entries, GEO_ID to DP05_0089PM
dtypes: object(358)
memory usage: 2.3+ MB


In [13]:
#acs2019_1.head(50)
acs2019_1['NAME'].value_counts().head(60)

Isabella County, Michigan               1
Osceola County, Florida                 1
Brazoria County, Texas                  1
Collier County, Florida                 1
Medina County, Ohio                     1
Wood County, Wisconsin                  1
Burke County, North Carolina            1
Catoosa County, Georgia                 1
King County, Washington                 1
Columbia County, Pennsylvania           1
Ventura County, California              1
Johnson County, Kansas                  1
Lonoke County, Arkansas                 1
Ector County, Texas                     1
Oklahoma County, Oklahoma               1
Davidson County, North Carolina         1
Stafford County, Virginia               1
Warren County, New Jersey               1
Trujillo Alto Municipio, Puerto Rico    1
Kankakee County, Illinois               1
Warren County, Ohio                     1
Kings County, New York                  1
Escambia County, Florida                1
Jefferson County, Alabama         